In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import re
import glob
import math
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
from datetime import datetime, timedelta
from collections import defaultdict
from multiprocessing import Manager
import sklearn

In [2]:
spark = (
    SparkSession.builder.appName('Fraud Imputation')
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/10/03 03:26:47 WARN Utils: Your hostname, DESKTOP-F216TKE resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/03 03:26:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/03 03:26:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
full_transactions = spark.read.parquet(".././data/curated/full_transaction3")

In [4]:
full_transactions.show()

+--------------+------------+-------+-----------+------------------+--------------------+------------------+-----+--------+------+-----------------+--------------------+--------------------+----+---------+--------------+
|order_datetime|merchant_abn|user_id|consumer_id|      dollar_value|            order_id|     consumer_name|state|postcode|gender|fraud_probability|       merchant_name|                tags|type|take_rate|merchant_fraud|
+--------------+------------+-------+-----------+------------------+--------------------+------------------+-----+--------+------+-----------------+--------------------+--------------------+----+---------+--------------+
|    2022-08-25| 63290521567|  11138|     401483|41.239626303220014|09f6132a-a6a8-47a...|   Veronica Nelson|   WA|    6519|Female|             NULL|Vehicula Pellente...|artist supply, craft|   a|     6.48|          NULL|
|    2022-08-25| 63465140133|  22285|     416278| 8.815286645396842|f4fc1a3b-a5b1-4d7...|       Luke Greene|   WA|  

In [5]:
full_transactions.printSchema()

root
 |-- order_datetime: date (nullable = true)
 |-- merchant_abn: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- consumer_id: string (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- consumer_name: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- fraud_probability: double (nullable = true)
 |-- merchant_name: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- type: string (nullable = true)
 |-- take_rate: double (nullable = true)
 |-- merchant_fraud: string (nullable = true)



In [5]:
full_transactions = full_transactions.withColumn('merchant_fraud', F.col('merchant_fraud').cast(DoubleType()))

In [6]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

#transactions = spark.read.parquet("../data/curated/transactions.parquet")

transactions = full_transactions.where(F.col("merchant_fraud").isNotNull()).toPandas()

categories = full_transactions.select('tags').distinct().toPandas().dropna().astype("string")['tags'].to_list()

X = transactions.drop('merchant_fraud', axis=1)
y = transactions['merchant_fraud']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

columns = list(X.columns.values)

preprocessor = ColumnTransformer(
    transformers=[
        ('tags', Pipeline([
            ('onehot', OneHotEncoder(categories=[categories], sparse_output=False, handle_unknown='ignore')),
        ]), [columns.index('tags')]), # tags
        ('state', Pipeline([
            ('onehot', OneHotEncoder(categories=[['NSW', 'VIC', 'TAS', 'QLD', 'WA', 'SA', 'NT', 'ACT']], sparse_output=False, handle_unknown='ignore')),
        ]), [columns.index('state')]), # state
        ('dollar_value', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler()),
        ]), [columns.index('dollar_value')]), # dollar value
        ('take_rate', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler()),
        ]), [columns.index('take_rate')]) # take rate
    ]
)

knn = KNeighborsRegressor(n_neighbors=3, weights='distance', p=1)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('knn', knn)
])

pipeline.fit(X_train, y_train)

# Display graph of predicted vs actual
pipeline.predict(X_test)
print(f"RMSE: {root_mean_squared_error(y_test, pipeline.predict(X_test))}")

RMSE: 2.6936593239469953


In [7]:
pipeline.fit(X, y)

sdf = pd.read_parquet('.././data/curated/full_transaction3', engine='fastparquet')



In [8]:
sdf.head()

,order_datetime,merchant_abn,user_id,consumer_id,dollar_value,order_id,consumer_name,state,postcode,gender,fraud_probability,merchant_name,tags,type,take_rate,merchant_fraud
0,2022-08-25,63290521567,11138,401483,41.239626,09f6132a-a6a8-47a2-a03e-383ddbf5feaa,Veronica Nelson,WA,6519,Female,NaN,Vehicula Pellentesque Corporation,"artist supply, craft",a,6.48,None
1,2022-08-25,63465140133,22285,416278,8.815287,f4fc1a3b-a5b1-4d75-b459-cf9bc12e1347,Luke Greene,WA,6170,Male,NaN,Vitae Odio Limited,"digital goods, books, movies, music",b,3.68,None
2,2022-08-25,21359184622,11139,505718,66.463016,b1a52524-b6e6-42a5-9726-687033350acc,Monica Garrett,NSW,2281,Female,NaN,Sit Amet PC,"motor vehicle supplies, new parts",b,3.60,None
3,2022-08-25,75034515922,22285,416278,19.654996,ba240514-5dea-4059-ac35-5917d4853188,Luke Greene,WA,6170,Male,NaN,Ac Eleifend Corp.,"digital goods, books, movies, music",a,6.22,None
4,2022-08-25,96334476428,11140,813297,6.383881,3f1c61aa-6b5b-4c8d-b981-f23029087927,Benjamin Schneider,VIC,3677,Male,NaN,Sed Est Ltd,"bicycle,",b,3.66,None


In [9]:
sdf_to_predict = sdf[sdf['merchant_fraud'].isnull()]

: 

In [ ]:
rows_with_na = df[df['A'].isnull()]

In [8]:
mask = sdf['merchant_fraud'].isna()

In [9]:
print(mask)

0           True
1           True
2           True
3           True
4           True
            ... 
13614849    True
13614850    True
13614851    True
13614852    True
13614853    True
Name: merchant_fraud, Length: 13614854, dtype: bool


In [10]:
sdf_to_predict = sdf.loc[mask, :]

: 

: 

In [9]:



X_to_predict = sdf_to_predict.drop('merchant_fraud', axis=1)

sdf.loc[mask, 'merchant_fraud'] = pipeline.predict(X_to_predict)


: 

In [ ]:
sdf.to_parquet(".././data/curated/transactions_merchantfraud", engine='pyarrow')